In [19]:
#import all the required libraries
import os
import numpy as np
import pandas as pd1
import shutil
import csv
import subprocess

In [15]:
#to create vivado project and add design files

#store all the files present in folder design_modules into list under variable design_files
design_files=os.listdir("design_modules")

#Convert the list of design files to a space-separated string
design_files_str = " ".join(design_files)

#pass the design_files_str of python as the argument into the tcl_add.tcl file
command="vivado -mode batch -source ./tcl_add.tcl -tclargs {}".format(design_files_str)
print("Executing command:", command)

# Use subprocess.run to capture output and errors
result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)

# Print the output and errors
print("Output:", result.stdout)
print("Errors:", result.stderr)

# Print the return code
print("Return code:", result.returncode)

Executing command: vivado -mode batch -source ./tcl_add.tcl -tclargs design1.v design2.v design3.v design4.v design5.v design6.v design7.v design8.v design9.v
Output: 
****** Vivado v2018.2 (64-bit)
  **** SW Build 2258646 on Thu Jun 14 20:02:38 MDT 2018
  **** IP Build 2256618 on Thu Jun 14 22:10:49 MDT 2018
    ** Copyright 1986-2018 Xilinx, Inc. All Rights Reserved.

source ./tcl_add.tcl
# create_project -force [pwd]/vivadoscript/Project_1  -part xc7a200tfbg676-2 
# set_property board_part xilinx.com:ac701:part0:1.4 [current_project]
# foreach file $argv {
#     add_files -norecurse [pwd]/modules/$file
# }
# update_compile_order -fileset sources_1
# close_project
INFO: [Common 17-206] Exiting Vivado at Tue Feb 13 21:35:29 2024...

Errors: 
Return code: 0


In [16]:
#to open the vivado project and synthesize and implement each file

#store all the files present in folder design_modules into list under variable design_files
design_files = os.listdir("design_modules")

# Convert the list of design files to a space-separated string and remove the last two string (.v) from the string
design_files_str = " ".join(['"' + file[:-2] + '"' for file in design_files])


#pass the design_files_str of python as the argument into the tcl_run.tcl file
command="vivado -mode batch -source ./tcl_run.tcl -tclargs {}".format(design_files_str)
print("Executing command:", command)

# Use subprocess.run to capture output and errors
result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)

# Print the output and errors
print("Output:", result.stdout)
print("Errors:", result.stderr)

# Print the return code
print("Return code:", result.returncode)


Executing command: vivado -mode batch -source ./tcl_run.tcl -tclargs "design1" "design2" "design3" "design4" "design5" "design6" "design7" "design8" "design9"
Output: 
****** Vivado v2018.2 (64-bit)
  **** SW Build 2258646 on Thu Jun 14 20:02:38 MDT 2018
  **** IP Build 2256618 on Thu Jun 14 22:10:49 MDT 2018
    ** Copyright 1986-2018 Xilinx, Inc. All Rights Reserved.

source ./tcl_run.tcl
# open_project [pwd]/vivadoscript/Project_1.xpr  
Scanning sources...
Finished scanning sources
# set_property source_mgmt_mode None [current_project]  
# update_compile_order -fileset sources_1	
# foreach file $argv {
# set_property top $file [current_fileset]
# set_property source_mgmt_mode None [current_project]
# update_compile_order -fileset sources_1
# 
# # reset and launch the synthesis and implementation. THe process waits till next command is executed
# 
# reset_run synth_1  
# launch_runs synth_1 -jobs 12
# wait_on_run synth_1
# 
# reset_run impl_1
# launch_runs impl_1 -jobs 12
# wait_on_r

In [20]:
#generate all_design_data.csv file that stores all the required timing,power and utilization information
txt_files_directory = "./Impl"
#store all the files present in folder design_modules into list under variable design_files
design_files = os.listdir("design_modules")

# remove (.v) from the list data above
designs = [file[:-2] for file in design_files]  

# Lists to store extracted data
DatapathDelay = []
logic_delay = []
routing_delay = []

file_name = []
DynamicPower = []
static_power = []
signal_power = []
logic_power = []
total_power = []

f7_muxes = []
lut_as_memory = []
lut_as_logic = []
slice_lut = []

#list to store all the data
all_design_data = []

#for each design file using for loop
for design in designs:
    # Define filenames for power, timing, and utilization
    power_file = os.path.join(txt_files_directory, f"{design}_power.txt")
    timing_file = os.path.join(txt_files_directory, f"{design}_timing.txt")
    utilization_file = os.path.join(txt_files_directory, f"{design}_utilization.txt")

    # Extract information from timing file
    
    with open(timing_file) as f:
        lines = f.read()
        timing = lines.split("\n")
        
        timing_info = [line for line in timing if "Data Path Delay" in line]
        dpd = float(timing_info[0].split()[3][:-2])
        DatapathDelay.append(dpd)
        

        timing_info = [line for line in timing if "logic" in line]
        dpd = float(timing_info[0].split()[5][:-2])
        logic_delay.append(dpd)
        

        timing_info = [line for line in timing if "route" in line]
        dpd = float(timing_info[0].split()[8][:-2])
        routing_delay.append(dpd)
        

    # Extract information from power file

    with open(power_file) as f:
        lines = f.read()
        power = lines.split("\n")
        
        power_info = [line for line in power if "Dynamic" in line]
        dpd = float(power_info[0].split()[4])
        DynamicPower.append(dpd)


        power_info = [line for line in power if "Static" in line]
        dpd = float(power_info[0].split()[5])
        static_power.append(dpd)


        power_info = [line for line in power if "Signals" in line]
        dpd = float(power_info[0].split()[3])
        signal_power.append(dpd)
        

        power_info = [line for line in power if "Logic" in line]
        dpd = float(power_info[0].split()[4])
        logic_power.append(dpd)
        

        power_info = [line for line in power if "Total" in line]
        dpd = float(power_info[0].split()[6])
        total_power.append(dpd)
        

    # Extract information from utilization file

    with open(utilization_file) as f:
        lines = f.read()
        utilization = lines.split("\n")
        
        utilization_info = [line for line in utilization if "F7" in line]
        dpd = float(utilization_info[0].split()[4])
        f7_muxes.append(dpd)
        

        utilization_info = [line for line in utilization if "Memory" in line]
        dpd = float(utilization_info[1].split()[5])
        lut_as_memory.append(dpd)
        

        utilization_info = [line for line in utilization if "Logic" in line]
        dpd = float(utilization_info[3].split()[5])
        lut_as_logic.append(dpd)
        

        utilization_info = [line for line in utilization if "Slice" in line]
        dpd = float(utilization_info[3].split()[4])
        slice_lut.append(dpd)
        

        design_data = {
        'Design': design,
        'DatapathDelay': DatapathDelay[-1],
        'LogicDelay': logic_delay[-1],
        'RoutingDelay': routing_delay[-1],
        'DynamicPower': DynamicPower[-1],
        'StaticPower': static_power[-1],
        'SignalPower': signal_power[-1],
        'LogicPower': logic_power[-1],
        'TotalPower': total_power[-1],
        'F7Muxes': f7_muxes[-1],
        'LutAsMemory': lut_as_memory[-1],
        'LutAsLogic': lut_as_logic[-1],
        'SliceLut': slice_lut[-1],
    }

    #append the above data into all_design_data
    all_design_data.append(design_data)

#generate a csv file 
csv_filename = "result.csv"

#open the csv_filename and write the data in csv file under the specific column
with open(csv_filename, mode='w', newline='') as csv_file:
    fieldnames = ['Design', 'DatapathDelay', 'LogicDelay', 'RoutingDelay', 'DynamicPower', 'StaticPower', 'SignalPower', 'LogicPower', 'TotalPower', 'F7Muxes', 'LutAsMemory', 'LutAsLogic', 'SliceLut']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    # Write header row
    writer.writeheader()

    # Write data rows
    writer.writerows(all_design_data)